In [ ]:
#TODOS make env object that uses julia solver
# details to figure out
# - how to set params inside julia (make sure changes to globals are persistent)
#   - apparently it's not so you'll need to figure out how to pass these params properly
# - check that discount across environment steps is consistent (e^-nu*(env_step_size))
# - verify the env with lots of runs (how to do this exactly call julia )

In [59]:
import julia
jl = julia.Julia(compiled_modules=False)
from julia import Main

In [60]:
import time

In [87]:
kappa = 0.001
BETA = 0.1
D = 0.1
PHI = 2*D+np.sqrt(BETA+4*D**2)#0.7#1.2
NU = 0.99

In [96]:
Main.xs = [1, 2, 3]
Main.eval("sin.(xs)")
Main.include("juliaScript.jl")
#Main.eval("helloWorld()")

<PyCall.jlwrap envStep>

In [98]:
u0=np.zeros(6)
loc = np.array2string(u0, separator=",")
start = time.time()
u_new,penalty = Main.eval(f"envStep({kappa}, {BETA}, {D}, {PHI}, {NU},{loc},1.)")
end = time.time()
end-start

0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2


0.0049936771392822266

In [45]:
start = time.time()
loc = np.array2string(u_new, separator=",")
u_new,penalty = Main.eval(f"envStep(0.5,{loc},1.)")
end = time.time()
end-start

0.005311489105224609

In [40]:
np.array2string(np.zeros(6), separator=",")

'[0.,0.,0.,0.,0.,0.]'

In [14]:
Main.eval("envStep()")

array([[ 0.00000000e+00,  1.42561928e-03,  3.51225956e-04, ...,
        -2.59398208e-03,  2.19019251e-04, -8.06107769e-04],
       [ 0.00000000e+00,  1.70797811e-05, -2.13292755e-03, ...,
         1.51217458e-02,  1.34861243e-02,  1.11173161e-02],
       [ 0.00000000e+00, -1.20205671e-02, -8.35756401e-03, ...,
         1.93549040e-01,  1.85147238e-01,  2.09788656e-01],
       [ 0.00000000e+00,  4.37356163e-02,  1.03682498e-01, ...,
        -1.77824951e-01, -1.41526248e-01, -1.89705977e-01],
       [ 0.00000000e+00,  1.20875809e-02,  5.11622853e-03, ...,
         1.13476351e-01,  6.53659801e-02,  4.25220043e-02],
       [ 0.00000000e+00,  1.20205671e-02,  8.35756401e-03, ...,
        -1.93549040e-01, -1.85147238e-01, -2.09788656e-01]])

In [ ]:
assert(1==0)

In [7]:
import sdeint
import numpy as np
#import random as rand
import matplotlib.pyplot as plt
#from itertools import accumulate
from scipy.optimize import curve_fit
from synthetic_env import synthetic_env
from naive_agents import DoNothing, FixedPhi

In [8]:
def f(u, t):
    #print(u)
    return np.array([
        u[2]*u[0] + u[3]*u[1] - PHI * u[0],
        u[4]*u[0] + u[5]*u[1] - PHI * u[1],
        -u[2],
        -u[3],
        -u[4],
        -u[5]
    ])

def g(u, t):
    return np.array([
        [np.sqrt(kappa), 0, 0, 0, 0],
        [0, np.sqrt(kappa), 0, 0, 0],
        [0, 0, np.sqrt(D), 0, 0],
        [0, 0, 0, np.sqrt(D), np.sqrt(2) * np.sqrt(D)],
        [0, 0, 0, np.sqrt(D), -np.sqrt(2) * np.sqrt(D)],
        [0, 0, -np.sqrt(D), 0, 0]
    ])


kappa = 0.001
BETA = 0.1
D=0.1
PHI=2*D+np.sqrt(BETA+4*D**2)#0.7#1.2

In [9]:
# Run the SDEs 
u0 = np.zeros(6)
tspan = np.arange(0.0, 1000.0,0.005)  # 10000 time units

start = time.time()
result = sdeint.itoint(f, g, u0, tspan)

separations = np.sum(result[:, :2]**2, axis=1)
separations = np.sqrt(separations)
end = time.time()
end-start

28.54189443588257

In [23]:
# Run the SDEs 
u0 = np.zeros(6)
tspan = np.arange(0.0, 1.0,0.005)  # 10000 time units

start = time.time()
result = sdeint.itoint(f, g, u0, tspan)

separations = np.sum(result[:, :2]**2, axis=1)
separations = np.sqrt(separations)
end = time.time()
end-start

0.037392616271972656

In [1]:
import time
from call_julia_class_test import Test_Solver

In [2]:
start = time.time()
x = Test_Solver()
end = time.time()
end-start

18.584166288375854

In [3]:
y = Test_Solver()

In [6]:
state,penalty = x.solve_flow(0.1)

In [8]:
type(state)

numpy.ndarray

In [5]:
y.solve_flow(10.2)

(array([ 0.0162689 , -0.00921762, -0.32855211, -0.38507772,  0.3229145 ,
         0.32855211]),
 0.0010260361687719568)